In [6]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



def getPrediction(OverallQual, slope, intercept):
    return slope * OverallQual + intercept

def getRelativeError(prediction, actual):
    return abs(prediction - actual) / actual * 100

def getAccuracy(df, slope, intercept):
    # Gather the average relative errors
    averageRelErrors = []
    for i in range(len(df['SalePrice'])):
        SalePrice = df.iloc[i]['SalePrice']
        OverallQual = df.iloc[i]['OverallQual']
        prediction = getPrediction(OverallQual, slope, intercept)
        relativeError = getRelativeError(prediction, SalePrice)
        averageRelErrors.append(relativeError)
    return sum(averageRelErrors) / len(averageRelErrors)



# Select predictor variable ("OverallQual") and target variable ("SalePrice")
X = df[['OverallQual']]
y = df['SalePrice']

# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features (optional but can be beneficial for linear regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Get the slope and intercept from the model
slope = model.coef_[0]
intercept = model.intercept_

# Evaluate the model using the getAccuracy function
accuracy = getAccuracy(X_test, slope, intercept)
print("Average Relative Error:", accuracy)


KeyError: 'SalePrice'